<a href="https://colab.research.google.com/github/WillN202/NLU_CW/blob/main/NLU_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NLU Task 2

In [2]:
!pip install torchmetrics
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 10.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchmetrics
import os.path
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import string
import re
TRAINING_DATASET_LOCATION = "/content/drive/MyDrive/train.csv"
DEV_DATASET_LOCATION = "/content/drive/MyDrive/dev.csv"
TEST_DATASET_LOCATION = "/content/drive/MyDrive/AV_trial.csv"
WORD2VEC_EMBEDDINGS = "/content/drive/MyDrive/word2vec_embeddings.model"
EPOCHS = 4

DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(DEVICE)
torch.set_default_device(DEVICE)

cuda


In [4]:
def generic_preprocessor(sentence):
  sentence = sentence.lower()

  return sentence

class AVDataset(Dataset):
  def __init__(self, csv_file, pre_processor=None):
    self.samples = pd.read_csv(csv_file)
    self.pre_processor = pre_processor

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, index):
    sample = self.samples.iloc[index]
    sample[0] = re.sub(f"[{re.escape(string.punctuation)}]", "", sample[0])
    sample[0] = re.sub(f"[{re.escape(string.punctuation)}]", "", sample[1])
    sample_text = f"{sample[0]} <sep> {sample[1]}"
    return (self.pre_processor(sample_text), sample[2])




In [5]:
training_samples = AVDataset(TRAINING_DATASET_LOCATION, pre_processor=generic_preprocessor)
dev_samples = AVDataset(DEV_DATASET_LOCATION, pre_processor=generic_preprocessor)
test_samples = AVDataset(TEST_DATASET_LOCATION, pre_processor=generic_preprocessor)
training_loader = DataLoader(training_samples, batch_size=64, shuffle=True, generator=torch.Generator(device=DEVICE))
dev_loader = DataLoader(dev_samples, batch_size=64, shuffle=True, generator=torch.Generator(device=DEVICE))
test_loader =  DataLoader(test_samples, batch_size=64, shuffle=True, generator=torch.Generator(device=DEVICE))

In [6]:
# TODO - Fix reading from saved embeddings
phrases = pd.read_csv(TRAINING_DATASET_LOCATION)
phrases = phrases.loc[:, "text_1":"text_2"].to_numpy().flatten().tolist()
phrases = [re.sub(f"[{re.escape(string.punctuation)}]", "", str(phrase)).split() for phrase in phrases]

embeddings = Word2Vec(sentences=phrases, workers=100, min_count=1, vector_size=256)
embeddings.wv["<UNK>"] = np.random.rand(256)
embeddings.wv["<sep>"] = np.random.rand(256)
embeddings.wv["<pad>"] = np.random.rand(256)
# embeddings.save("/content/drive/MyDrive/word2vec_embeddings.model")
# WORD2VEC_EMBEDDINGS = "/content/drive/MyDrive/word2vec_embeddings.model"

#embeddings = Word2Vec.load(WORD2VEC_EMBEDDINGS)


In [7]:
class LstmRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab):
    super(LstmRNN, self).__init__()
    self.linear_layer = torch.nn.Sequential(
        torch.nn.Linear(hidden_size, output_size),
    )
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    # TODO -> Consider adding dropout for higher levels
    self.LSTM_Layer = torch.nn.LSTM(embedding_size, hidden_size, batch_first=True)

  def forward(self, x):
    unk_embedding = self.vocab["<UNK>"]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    x = torch.tensor(x)

    embeddings = self.get_embedding(x)
    LSTM_values = self.LSTM_Layer(embeddings)[1]
    LSTM_values = LSTM_values[0][-1] #Get last hidden state(s)
    result = self.linear_layer(LSTM_values)
    return result

class LstmBidirectionalRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab):
    super(LstmBidirectionalRNN, self).__init__()
    self.linear_layer = torch.nn.Sequential(
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.LeakyLeakyReLU(),
        torch.nn.Dropout(p=0.1),
        torch.nn.Linear(hidden_size, output_size)
    )
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    # TODO -> Consider adding dropout for higher levels
    self.LSTM_Layer = torch.nn.LSTM(embedding_size, hidden_size, batch_first=True, bidirectional=True)

  def forward(self, x):
    unk_embedding = self.vocab["<UNK>"]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    x = torch.tensor(x)

    embeddings = self.get_embedding(x)
    LSTM_values = self.LSTM_Layer(embeddings)[1]
    LSTM_values = LSTM_values[0][-1] #Get last hidden state(s)
    result = self.linear_layer(LSTM_values)
    return result


class BaseGruRNN(torch.nn.Module):
  def __init__(self, createModule):
    super(createModule, self).__init__()

  def forward(self, x):
    unk_embedding = self.vocab["<UNK>"]
    x = [re.sub(f"[{re.escape(string.punctuation)}]", "", sentence) for sentence in x]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    x = torch.tensor(x)

    embeddings = self.get_embedding(x)
    GRU_values = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_values[0] #Get last hidden state(s)
    result = self.linear_layer(GRU_values)
    return result

class OneLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1):
      super(OneLinearLayerGruRNN, self).__init__()
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, output_size),
      )
      self.vocab = vocab
      self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
      self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1)

class TwoLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1):
      super(OneLinearLayerGruRNN, self).__init__()
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )
      self.vocab = vocab
      self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
      self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1)

class ThreeLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1):
      super(OneLinearLayerGruRNN, self).__init__()
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )
      self.vocab = vocab
      self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
      self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1)

class FourLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1):
      super(OneLinearLayerGruRNN, self).__init__()
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )
      self.vocab = vocab
      self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
      self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1)

class BaseGruBidirectionalRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab):
    super(GruBidirectionalRNN, self).__init__()
    self.linear_layer = torch.nn.Sequential(
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.LeakyLeakyReLU(),
        torch.nn.Dropout(p=0.1),
        torch.nn.Linear(hidden_size, output_size)
    )
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    # TODO -> Consider adding dropout for higher levels
    self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, bidirectional=True, num_layers=3)

  def forward(self, x):
    unk_embedding = self.vocab["<UNK>"]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    x = torch.tensor(x)

    embeddings = self.get_embedding(x)
    GRU_values = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_values[-1] #Get last hidden state(s)
    result = self.linear_layer(GRU_values)
    return result

coded_embeddings = torch.FloatTensor(embeddings.wv.vectors).to(DEVICE)
vocab = embeddings.wv.key_to_index


## Model Training

In [ ]:
# TODO figure out why default device isn't working
model = GruRNN(256, 1, 256, coded_embeddings, vocab).to(DEVICE)
loss_function = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
torch.set_grad_enabled(True)

print(f"Epochs: {EPOCHS}")

for epoch in range(0, EPOCHS):
    model.train()
    running_loss = 0.0

    # Training
    for index, value in enumerate(training_loader):
        optimizer.zero_grad()
        data, labels = value
        labels = labels.reshape(-1,1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(DEVICE)
        outputs = model(data)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation to ensure the model is learning
    model.eval()
    torch.set_grad_enabled(False)
    running_loss = 0
    num_correct = 0
    for index, value in enumerate(dev_loader):
        data, labels = value
        labels = labels.reshape(-1,1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(DEVICE)

        outputs = model(data)
        loss = loss_function(outputs, labels)
        running_loss += loss

        for answer, standard in zip(outputs, labels):
            normalised_answer = torch.round(torch.sigmoid(answer[0]))
            num_correct += 1 if normalised_answer == standard[0] else 0

    torch.set_grad_enabled(True)
    print(f"---------------------EPOCH {epoch+1} / {EPOCHS}---------------------")
    print(f"Batch Loss {running_loss / len(dev_loader)}")
    print(f"Accuracy {num_correct / len(dev_loader.dataset)}")



Epochs: 4
---------------------EPOCH 1 / 4---------------------
Batch Loss 0.691735029220581
Accuracy 0.5165
---------------------EPOCH 2 / 4---------------------
Batch Loss 0.6915404796600342
Accuracy 0.5196666666666667
---------------------EPOCH 3 / 4---------------------
Batch Loss 0.6917790174484253
Accuracy 0.5156666666666667
---------------------EPOCH 4 / 4---------------------
Batch Loss 0.6926555633544922
Accuracy 0.5183333333333333


## Model Testing

In [ ]:
model.eval()
torch.set_grad_enabled(False)
running_loss = 0
num_correct = 0

loader = test_loader

for index, value in enumerate(loader):
    data, labels = value
    labels = labels.reshape(-1,1)
    labels = labels.type(torch.FloatTensor)
    labels = labels.to(DEVICE)

    outputs = model(data)
    loss = loss_function(outputs, labels)
    running_loss += loss

    for answer, standard in zip(outputs, labels):
        normalised_answer = torch.round(torch.sigmoid(answer[0]))
        num_correct += 1 if normalised_answer == standard[0] else 0

torch.set_grad_enabled(True)
print(f"Accuracy {num_correct / len(loader.dataset)}")

Accuracy 0.56


In [ ]:
# Results for 1-2-3
class BestSoFarGruRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab):
    super(GruRNN, self).__init__()
    self.linear_layer = torch.nn.Sequential(
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.Dropout(p=0.4),
        torch.nn.Linear(hidden_size, output_size)
    )
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    # TODO -> Consider adding dropout for higher levels
    self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True)

  def forward(self, x):
    unk_embedding = self.vocab["<UNK>"]
    x = [re.sub(f"[{re.escape(string.punctuation)}]", "", sentence) for sentence in x]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    x = torch.tensor(x)

    embeddings = self.get_embedding(x)
    GRU_values = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_values[0] #Get last hidden state(s)
    result = self.linear_layer(GRU_values)
    return result


# Used for 4
class EvenBetterGruRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab):
    super(GruRNN, self).__init__()
    self.linear_layer = torch.nn.Sequential(
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(p=0.1),
        torch.nn.Linear(hidden_size, output_size)
    )
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    # TODO -> Consider adding dropout for higher levels
    self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True)

  def forward(self, x):
    unk_embedding = self.vocab["<UNK>"]
    x = [re.sub(f"[{re.escape(string.punctuation)}]", "", sentence) for sentence in x]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    x = torch.tensor(x)

    embeddings = self.get_embedding(x)
    GRU_values = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_values[0] #Get last hidden state(s)
    result = self.linear_layer(GRU_values)
    return result

# Same embeddings used across the model, with punctuation removal
embeddings.save("/content/drive/MyDrive/word2vec_embeddings.model")
torch.save(model.state_dict(), "/content/drive/MyDrive/unidirectional_gru_5.model")